In [50]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
import os

# Chunking van Markdown

## 1. Doelstelling

RAG komt erop neer dat een LLM wordt gevoed met extra context die relevant is voor een specifieke vraag, zodat het model beter in staat is om die vraag te beantwoorden. Die context kan echter niet onbeperkt groot zijn, om twee belangrijke redenen:

1. De meeste LLM’s hanteren een limiet op de grootte van de invoer die ze kunnen verwerken. Dit betekent dat zowel de vraag als de bijhorende context binnen een vast contextvenster moeten passen.
2. LLM’s zijn gevoelig voor het *lost in the middle*-fenomeen: ze besteden meer aandacht aan het begin en het einde van de invoer, en minder aan het minder van de invoer.

Voor deze redenen is het nodig om de tekst afkomstig uiy de PDF-bestanden te verdelen in *chunks* (= brokken). 

Om tekst op te splitsen in chunks, bestaan er uiteenlopende chunkingstechnieken — gaande van zeer eenvoudige aanpakken, waarbij elke chunk een vaste lengte $n$ heeft en er desnoods midden in een zin gesneden wordt, tot complexere methoden waarbij embeddingsmodellen worden gebruikt om zinnen semantisch te groeperen op basis van hun betekenis. Onze keuze om PDF-bestanden om te zetten naar Markdown is niet blindelings gemaakt. We hebben bewust gezocht naar een compromis waarbij de semantiek tijdens het chunken behouden blijft, zonder dat we een bijzonder computationeel intensieve methode hoeven te gebruiken.

Marker (zie dit [Notebook](Tekstextractie_PDF.ipynb)) probeert de structuur van de originele PDF te behouden tijdens de omzetting naar Markdown. We geloven dat in die structuur vaak ook de onderliggende semantiek vervat zit, aangezien elementen die inhoudelijk samenhangen doorgaans ook op een gelijkaardige manier worden gestructureerd. Daarom kiezen we ervoor om te chunken op basis van de Markdown-structuur van het bestand.

[Langchain](https://python.langchain.com/docs/introduction/) biedt zogenaamde *Text splitters* daan, die tekst opdelen in chunks volgens bepaalde strategieën. Eén van deze splitters is de `MarkdownHeaderTextSplitter`, die specifiek ontworpen werd om Markdown-tekst te verdelen op basis van de structuur van de headers. In dit Notebook bekijken we hoe deze splitter precies werkt.

## 2. Uittesten

In [29]:
# Defining path to Markdown file
file_path = './markdowns/Marker/Studiewijzers/Studiewijzer_Infrastructure_Automation.md'

### 2.1. `MarkdownHeaderTextSplitter`

In [71]:
# Opening Markdown file
with open(file_path, 'r', encoding='utf-8') as file:
    md_file = file.read()

In [73]:
# Defining splitter and creating chunks

## Defining headers which will be used to split
headers_to_split_on = [
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
    ("####", "h4"),
]

md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on) # Creating splitter
docs = md_splitter.split_text(md_file) # Chunking and returning chunks in the form of Langchain documents

In [77]:
# Printing chunks
for i, doc in enumerate(docs):
    print(f'--------- CHUNK {i + 1} ---------')
    print(doc.page_content)
    print()

--------- CHUNK 1 ---------
- 1. Doel en plaats van de cursus in het curriculum
- 2. Leerdoelen en competenties
- 3. Leerinhoud
- 4. Leermateriaal
- 5. Werkvormen
- 6. Werk- en leeraanwijzingen
- 7. Studiebegeleiding
- 8. Planning
- 9. Evaluatie

--------- CHUNK 2 ---------
Deze studiewijzer geldt zowel voor de reguliere studenten als de studenten afstandsleren (TIAO). Waar nodig wordt duidelijk onderscheid gemaakt tussen zaken die enkel voor één van deze twee groepen gelden. Wat betreft praktische afspraken, regelingen, verwachtingen, enz. in verband met deze cursus zijn de enige geldige bronnen van informatie:  
- De studiecheche van dit opleidingsonderdeel, te bekijken via de Chamilo- cursus of [https://hogent.be/studieches/](https://hogent.be/studiefiches/) [\(https://hogent.be/studieches/\)](https://hogent.be/studiefiches/);
- Deze studiewijzer;
- Documenten op Chamilo;
- Aankondigingen op Chamilo—deze worden ook telkens per e-mail naar de betrokken studenten gestuurd;
- Het Onder